<a href="https://colab.research.google.com/github/shubhangkhare/Learnings/blob/main/List%20of%20Python%20Libraries%20in%20Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone Repository

In [1]:
!git clone https://github.com/shubhangkhare/Learnings -q

# Extract python libraries from ipynb files

## Function

### CASE 1: import transformers, langchain #Some comment

In [12]:
import re

# Example usage
string = "import transformers, langchain #Some comment"

# Define the regular expression pattern to match library names
pattern = r'(?i)\b(?:import|from)\s+([\w\s,]+)\b'

# Find all matches of the pattern in the string
matches = re.findall(pattern, string)

# Extracted libraries
extracted_libraries = [library.strip() for library in matches[0].split(',')]

print(extracted_libraries)


['transformers', 'langchain']


### CASE 2: from transformers import AutoTokenizer, pipeline

In [14]:
import re

# Example usage
string = "from transformers.emb import AutoTokenizer, pipeline # Some comment"

# Define the regular expression pattern to match library name
pattern = r'(?i)\bfrom\s+([\w.]+)'

# Find the match of the pattern in the string
match = re.search(pattern, string)

# Extracted library
extracted_library = match.group(1)

print(extracted_library)

transformers.emb


### CASE 3: !pip install transformers langchain -q # Some comment

In [40]:
import re

# Example usage
string = "!pip install langchain tiktoken chromadb pypdf# Some comment"

# Extracted libraries
extracted_libraries = re.findall(r'!pip install\s+([\w\s,-]+)', string)

# Split the extracted libraries by whitespace and filter out non-library elements
extracted_libraries = [library.strip() for library in extracted_libraries[0].split() if not library.startswith('-')]

print(extracted_libraries)


['langchain', 'tiktoken', 'chromadb', 'pypdf']


### Remove -q -p from code

In [63]:
import re

# Example usage
string = "!pip -q install accelerate bitsandbytes"

# Remove expressions like '-q' or '-p'
string = re.sub(r'\s+-[qp]\s+', ' ', string)

print(string)


!pip install accelerate bitsandbytes


### Final Extractor Function

In [54]:
import nbformat
import pandas as pd

def extract_libraries_from_notebook(notebook_file, output_file):
    # Read the Jupyter Notebook file
    with open(notebook_file, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)

    # Extract the libraries from the notebook
    libraries = []
    for cell in nb['cells']:
        if cell['cell_type'] == 'code':
            lines = cell['source'].split('\n')
            for line in lines:
                line = line.strip()
                # Remove -q -p
                line = re.sub(r'\s+-[qp]\s+', ' ', line)
                #CASE 1: import transformers, langchain # Comment
                if line.startswith('import'):
                    # Define the regular expression pattern to match library names
                    pattern = r'(?i)\b(?:import|from)\s+([\w\s,]+)\b'
                    # Find all matches of the pattern in the string
                    matches = re.findall(pattern, line)
                    # Extracted libraries
                    extracted_libraries = [library.strip() for library in matches[0].split(',')]
                    libraries += extracted_libraries
                # CASE 2: from transformers.emb import AutoTokenizer, pipeline # Comment
                if line.startswith('from'):
                    # Define the regular expression pattern to match library name
                    pattern = r'(?i)\bfrom\s+([\w.]+)'
                    # Find the match of the pattern in the string
                    match = re.search(pattern, line)
                    # Extracted library
                    extracted_library = match.group(1)
                    if '.' in extracted_library:
                        extracted_library = extracted_library.split('.')[0]
                    libraries.append(extracted_library)
                # CASE 3: !pip install transformers langchain -q # Some comment
                if line.startswith('!pip install'):
                    # Extracted libraries
                    extracted_libraries = re.findall(r'!pip install\s+([\w\s,-]+)', string)
                    # Split the extracted libraries by whitespace and filter out non-library elements
                    extracted_libraries = [library.strip() for library in extracted_libraries[0].split() if not library.startswith('-')]
                    libraries += extracted_libraries

    # Remove duplicates from the library list
    libraries = list(set(libraries))

    # Create a DataFrame to store the libraries
    df = pd.DataFrame({'Library': libraries})

    # Save the libraries as an Excel file
    df.to_excel(output_file, index=False)

    print(f"Libraries extracted from the notebook and saved to '{output_file}'.")

### Example usage

In [55]:
notebook_file = '/content/Learnings/HF_Instructor_Embeddings_Chroma_DB_Multi_Doc_Retriever_LangChain_Part2.ipynb'
output_file = '/content/output_libraries.xlsx'
extract_libraries_from_notebook(notebook_file, output_file)

Libraries extracted from the notebook and saved to '/content/output_libraries.xlsx'.


### Check the extracted libraries

In [60]:
df.shape

(11, 1)

In [59]:
df = pd.read_excel(output_file)
df.head()

,Library
0,transformers
1,openai
2,textwrap
3,pypdf
4,langchain


In [53]:
df.head(11)

,Library
0,transformers
1,openai
2,textwrap
3,pypdf
4,langchain
5,os
6,requests
7,torch
8,tiktoken
9,InstructorEmbedding


(8, 1)

# Check Library is installed or not

In [84]:
from IPython import get_ipython

# Command in string format
command = "import openai"

# Execute the command using run_cell_magic
get_ipython().run_cell_magic('python', '', command)


Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ModuleNotFoundError: No module named 'openai'


CalledProcessError: ignored

In [83]:
np.zeros(4)

NameError: ignored

In [93]:
import pandas as pd
import pip
import pkg_resources
from IPython import get_ipython
from IPython.display import clear_output


def check_installation(library):
    try:
        command = 'import {0}'.format(library)
        # Execute the command using run_cell_magic
        get_ipython().run_cell_magic('python', '', command)
        return 'YES'
    except:
        return 'NO'

# Create a new column 'Installed' and populate it based on library installation status
df['Installed'] = df['Library'].apply(check_installation)
clear_output()
df.head(11)
# Save the updated DataFrame to a new Excel file
#df.to_excel('path/to/output_file.xlsx', index=False)

,Library,Installed
0,transformers,NO
1,openai,NO
2,textwrap,YES
3,pypdf,NO
4,langchain,NO
5,os,YES
6,requests,YES
7,torch,YES
8,tiktoken,NO
9,InstructorEmbedding,NO


# Get latest compatible Python Library Version

In [75]:
import xmlrpc.client
import time

# Function to get the latest version of a library from PyPI
def get_latest_version(library):
    pypi = xmlrpc.client.ServerProxy('https://pypi.org/pypi')
    releases = pypi.package_releases(library)
    time.sleep(1)  # Add a delay of 1 second

    if releases:
        latest_version = releases[0]
        return latest_version
    else:
        return None

# Example usage
df['Latest Version'] = df['Library'].apply(get_latest_version)
df.head()

,Library,Installed,Latest Version
0,transformers,No,4.30.2
1,openai,No,0.27.8
2,textwrap,No,None
3,pypdf,No,3.12.1
4,langchain,No,0.0.234


# PYPI URL

In [94]:
def pypi_url(library):
    url = 'https://pypi.org/project/{0}/'.format(library)
    return url

df['PyPI URL'] = df['Library'].apply(pypi_url)
df.head(11)

,Library,Installed,PyPI URL
0,transformers,NO,https://pypi.org/project/transformers/
1,openai,NO,https://pypi.org/project/openai/
2,textwrap,YES,https://pypi.org/project/textwrap/
3,pypdf,NO,https://pypi.org/project/pypdf/
4,langchain,NO,https://pypi.org/project/langchain/
5,os,YES,https://pypi.org/project/os/
6,requests,YES,https://pypi.org/project/requests/
7,torch,YES,https://pypi.org/project/torch/
8,tiktoken,NO,https://pypi.org/project/tiktoken/
9,InstructorEmbedding,NO,https://pypi.org/project/InstructorEmbedding/
